In [1]:
import numpy as np
import time 
import os, sys, traceback
import cPickle as pickle

%matplotlib inline
import matplotlib.pylab as plt

import numpy as np
from sigvisa import Sigvisa

from sigvisa.treegp.gp import GP, GPCov
from sigvisa.models.spatial_regression import SparseGP

%load_ext autoreload
%autoreload 1
%aimport sigvisa.models.spatial_regression.local_gp_ensemble 
%aimport sigvisa.learn.train_param_common

tpc = sigvisa.learn.train_param_common
lge = sigvisa.models.spatial_regression.local_gp_ensemble 




In [2]:
s = Sigvisa()
cursor = s.dbconn.cursor()
cursor.execute("select lon, lat, depth, mb from isc_origin where mb > 0")
V = np.array(cursor.fetchall())
cursor.close()
X = V[:, :3].copy()
y = V[:, 3].copy()

from sigvisa.utils.geog import dist_km
sta_lon = -110
sta_lat = 45

XX = np.array([(lon, lat, depth, dist_km((lon, lat), (sta_lon, sta_lat)), 0.0) for (lon, lat, depth) in X])
x0 = XX[0:1, :]


# Saving "tight" GP files with no Kinv data

In [3]:

yvar = np.random.rand(*y.shape)
basic_cov = GPCov(wfn_str="matern32", wfn_params=(1.0,), dfn_str="lld", dfn_params=(30.0, 30.0))

gp = GP(XX, y, y_obs_variances=yvar, 
        cov_main = basic_cov, noise_var=1.0,
        sparse_invert=False)

In [4]:
gp.save_trained_model("loose.gp", tight=False)
gp.save_trained_model("tight.gp", tight=True)

In [5]:
gp2 = GP(fname="loose.gp")

In [6]:
gp3 = GP(fname="tight.gp")

In [15]:
print gp.predict(x0)
print gp2.predict(x0)
print gp3.predict(x0)

3.22171800753
[ 3.22171801]
[ 3.22171801]


# Lazily loaded localGPs

In [5]:
cluster_centers = np.loadtxt("/home/dmoore/python/sigvisa/isc_clusters")
n_clusters = cluster_centers.shape[0]
basic_cov = GPCov(wfn_str="matern32", wfn_params=(1.0,), dfn_str="lld", dfn_params=(30.0, 30.0))
cluster_covs = [basic_cov.copy() for i in range(n_clusters)]
cluster_noise_vars = [1.0 for i in range(n_clusters)]

from sigvisa.learn.train_param_common import pre_featurizer
basisfn_str="sin1"
basisfn_str, featurizer_recovery, extract_dim = pre_featurizer(basisfn_str)


In [14]:
param_mean = np.zeros((4,))
param_cov = np.eye(4)
lgp_param = lge.LocalGPEnsemble(XX, y, 
                          cluster_centers=cluster_centers, 
                          cluster_covs = cluster_covs,
                          cluster_noise_vars = cluster_noise_vars,
                          basis=basisfn_str,
                          extract_dim=extract_dim,
                          featurizer_recover=featurizer_recovery,
                          prior_mean=param_mean,
                          prior_cov=param_cov)


In [15]:
print lgp_param.predict(x0)
print lgp_param.variance(x0, include_obs=True)

3.28529170792
1.08235174285


In [16]:
lgp_param.save_trained_model("test_lgp")

In [17]:
lgp_loaded = lge.load_lgp_ensemble("test_lgp")

In [18]:
print lgp_loaded

In [19]:
print lgp_loaded.local_gps

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [21]:
lgp_loaded.predict(x0)
print lgp_loaded.local_gps

[<sigvisa.treegp.gp.GP object at 0x7f6d542b6850>, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
